In [ ]:
import typing as t

from aiida import load_profile, orm

from aiida_common_workflows.workflows.afm import AfmCase, AfmWorkflow

_ = load_profile()

In [ ]:
from aiida_pseudo.groups.family.pseudo import PseudoPotentialFamily

structure = orm.StructureData()
structure.set_cell(
    [
        [14.9412827110, 0.0, 0.0],
        [0.0, 14.5091262213, 0.0],
        [0.0, 0.0, 10.0820001747],
    ]
)
structure.set_pbc((False, False, False))
structure.append_atom(symbols="C", position=(8.2766055186, 6.1177492411, 5.0595246063))
structure.append_atom(symbols="C", position=(8.8582363932, 7.3839854920, 5.0363959612))
structure.append_atom(symbols="C", position=(8.0523006568, 8.5208073256, 5.0178902940))
structure.append_atom(symbols="C", position=(6.6646772714, 8.3913773689, 5.0224782237))
structure.append_atom(symbols="C", position=(6.0830463468, 7.1251409580, 5.0456056988))
structure.append_atom(symbols="C", position=(6.8889822232, 5.9883191444, 5.0641114560))
structure.append_atom(symbols="H", position=(8.9093056269, 5.2253257080, 5.0739448106))
structure.append_atom(symbols="H", position=(9.9475591624, 7.4856249613, 5.0328185108))
structure.append_atom(symbols="H", position=(8.5089059407, 9.5148483491, 4.9998780126))
structure.append_atom(symbols="H", position=(6.0319773431, 9.2838011020, 5.0080584394))
structure.append_atom(symbols="H", position=(4.9937235476, 7.0235011287, 5.0491811392))
structure.append_atom(symbols="H", position=(6.4323774593, 4.9942778709, 5.0821221574))

tip = orm.StructureData()
tip.set_cell(
    [
        [20.0, 0.0, 0.0],
        [0.0, 20.0, 0.0],
        [0.0, 0.0, 20.0],
    ]
)
tip.set_pbc((False, False, False))
tip.append_atom(symbols="C", position=(0.0, 0.0, 1.15))
tip.append_atom(symbols="O", position=(0.0, 0.0, 0.0))

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])

pseudo_family = t.cast(PseudoPotentialFamily, orm.load_group(4))
C_pp = pseudo_family.get_pseudo("C")
H_pp = pseudo_family.get_pseudo("H")
O_pp = pseudo_family.get_pseudo("O")

In [ ]:
afm_params = {
    "PBC": False,
    "tip": "s",
    "klat": 0.3490127886809,
    "krad": 21.913190531846,
    "gridA": [14.9412827110, 0.0000000000, 0.0000000000],
    "gridB": [0.0000000000, 14.5091262213, 0.0000000000],
    "gridC": [0.0000000000, 0.0000000000, 10.0820001747],
    "sigma": 0.7,
    "charge": 0.0,
    "r0Probe": [0.0, 0.0, 2.97],
    "scanMax": [14.9412827110, 14.5091262213, 11],
    "scanMin": [0.0, 0.0, 8],
    "scanStep": [0.1, 0.1, 0.1],
    "Amplitude": 1.4,
    "probeType": "O",
    "f0Cantilever": 22352.5,
    "gridN": [-1, -1, -1],
}

In [ ]:
dft_params = {
    "geom": {
        "engines": {
            "relax": {
                "code": orm.load_code("pw-7.4@localhost"),
                "options": {
                    "resources": {
                        "num_machines": 1,
                    },
                    "max_wallclock_seconds": 43200,
                },
            },
        },
        "protocol": "fast",
        "relax_type": "positions",
    },
    "tip": {
        "engines": {
            "relax": {
                "code": orm.load_code("pw-7.4@localhost"),
                "options": {
                    "resources": {
                        "num_machines": 1,
                    },
                    "max_wallclock_seconds": 43200,
                },
            }
        },
        "protocol": "fast",
        "relax_type": "none",
    },
}

In [ ]:
pp_params = {
    "hartree_potential": {
        "engines": {
            "pp": {
                "code": orm.load_code("pp-7.4@localhost"),
                "options": {
                    "resources": {
                        "num_machines": 1,
                    },
                    "max_wallclock_seconds": 43200,
                },
            }
        },
        "quantity": "potential",
    },
    "charge_density": {
        "engines": {
            "pp": {
                "code": orm.load_code("pp-7.4@localhost"),
                "options": {
                    "resources": {
                        "num_machines": 1,
                    },
                    "max_wallclock_seconds": 43200,
                },
            }
        },
        "quantity": "charge_density",
    },
}


In [ ]:
wg = AfmWorkflow.build(
    engine="quantum_espresso",
    case=AfmCase.HARTREE.name,
    structure=structure,
    afm_params=afm_params,
    relax=False,
    dft_params=dft_params,
    pp_params=pp_params,
    tip=tip,
)

In [ ]:
wg

In [ ]:
wg.run()

In [ ]:
import base64

from IPython.display import HTML

fd: orm.FolderData
if wg.outputs.Q0_00K0_35.value:
    fd = wg.outputs.Q0_00K0_35.value
else:
    node = orm.load_node(wg.pk)
    fd = node.outputs.Q0_00K0_35

imgs = []

png_folder = "Amp1.40"

for obj in fd.list_objects(png_folder):
    if obj.name.endswith(".png"):
        with fd.open(f"{png_folder}/{obj.name}", "rb") as handle:
            data = handle.read()
            data64 = base64.b64encode(data).decode("utf-8")
            imgs.append(f"""
                <img
                    src="data:image/png;base64,{data64}"
                    style="max-width:150px; margin:5px;"
                />
            """)

HTML("".join(imgs))